In [2]:
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path
from datetime import datetime, timedelta
%matplotlib inline
import yfinance as yf
import mibian as mb

In [3]:
tsla_df = pd.read_pickle('tsla_merged.pickle')

In [5]:
df = tsla_df
df = df.drop(columns= ['LastPrice', 'Symbol', 'Volume', 'OpenInterest', 'Volume'])
#df['OptionKey'] =df['OptionKey'].map(lambda x: str(x)[:-10])


In [19]:
df['ExpirationDate'] =  pd.to_datetime(df['ExpirationDate'], infer_datetime_format=True)
df['DataDate'] =  pd.to_datetime(df['DataDate'], infer_datetime_format=True)
df['delta'] = df['ExpirationDate'] - df['DataDate']

df['exp?'] =df['ExpirationDate'] ==df['DataDate']
df['long_close'] = ((df['delta'] < np.timedelta64(14, 'D')) & (df['% out'] > -.1) & (df['% out'] < 0))
df['long_medium'] = ((df['delta'] < np.timedelta64(14, 'D')) & (df['% out'] > -.2) & (df['% out'] < -.1))
df['long_far'] = ((df['delta'] < np.timedelta64(14, 'D')) & (df['% out'] > -.3) & (df['% out'] < -.2))
df['short_close'] = ((df['delta'] < np.timedelta64(8, 'D')) & (df['% out'] > -.1) & (df['% out'] < 0))
df['short_medium'] = ((df['delta'] < np.timedelta64(8, 'D')) & (df['% out'] > -.2) & (df['% out'] < -.1))
df['short_far'] = ((df['delta'] < np.timedelta64(8, 'D')) & (df['% out'] > -.3) & (df['% out'] < -.2))

#df['collatarel'] = ((df['delta'] < np.timedelta64(14, 'D')) & (df['% out'] > -.4) & (df['% out'] < -.2))
df['Signal'] = np.random.randint(-1,2, df.shape[0])
df.loc[(df['Signal'] == 1) & (df['worth_it'] == True), 'execute']=df.loc[(df['Signal'] == 1) & (df['worth_it'] == True), 'AskPrice']
df['enter?'] = (df['worth_it'] == True) & (df['Signal'] == 1)
df.loc[df['enter?'] == True]
df

,OptionKey,ExpirationDate,AskPrice,BidPrice,PutCall,StrikePrice,UnderlyingPrice,DataDate,time delta,% out,...,Signal,execute,enter?,xp,long_close,long_medium,long_far,short_close,short_medium,short_far
2019-11-14,TSLA2019-11-15P135.00,2019-11-15,0.01,0.00,put,135.0,349.35,2019-11-14,1 days,-0.613568,...,-1,NaN,False,NaN,False,False,False,False,False,False
2019-11-13,TSLA2019-11-15P155.00,2019-11-15,0.05,0.00,put,155.0,346.11,2019-11-13,2 days,-0.552165,...,1,NaN,False,NaN,False,False,False,False,False,False
2019-11-13,TSLA2019-11-15P165.00,2019-11-15,0.01,0.00,put,165.0,346.11,2019-11-13,2 days,-0.523273,...,1,NaN,False,0.01,False,False,False,False,False,False
2019-11-15,TSLA2019-11-15P165.00,2019-11-15,0.01,0.00,put,165.0,352.17,2019-11-15,0 days,-0.531476,...,0,NaN,False,0.01,False,False,False,False,False,False
2019-11-15,TSLA2019-11-15P170.00,2019-11-15,0.01,0.00,put,170.0,352.17,2019-11-15,0 days,-0.517279,...,-1,NaN,False,0.01,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-19,TSLA2019-12-27P475.00,2019-12-27,118.35,114.00,put,475.0,359.52,2019-11-19,38 days,0.321206,...,0,NaN,False,NaN,False,False,False,False,False,False
2019-11-22,TSLA2019-12-27P475.00,2019-12-27,143.75,139.75,put,475.0,333.04,2019-11-22,35 days,0.426255,...,0,NaN,False,NaN,False,False,False,False,False,False
2019-11-25,TSLA2019-12-27P475.00,2019-12-27,140.50,136.35,put,475.0,336.34,2019-11-25,32 days,0.412261,...,1,NaN,False,NaN,False,False,False,False,False,False
2019-12-18,TSLA2019-12-27P490.00,2019-12-27,97.60,96.30,put,490.0,393.15,2019-12-18,9 days,0.246344,...,0,NaN,False,NaN,False,False,False,False,False,False


In [14]:
df_table=df.loc[:, ['OptionKey', 'BidPrice']]
df_table=df_table.reset_index()
df_table.columns=['PriceDate', 'OptionKey', 'RebuyPrice']
df_table['RebuyPrice'].nunique()

1259

In [86]:
#msft_df_table['PriceDate'] =  pd.to_datetime(msft_df_table['PriceDate'], infer_datetime_format=True)
#msft_result = pd.merge(msft_df, msft_df_table, left_on=['ExpirationDate', 'OptionKey'], right_on=['PriceDate', 'OptionKey'], how='left')
#msft_result

In [87]:
#ok=msft_result.columns.values.tolist()
#for column_name in ok:
 #  print(msft_result['RebuyPrice'].unique())

In [88]:
#exp = (df.loc[df['exp?'] == True], 

In [15]:
xp = df
xp.loc[xp['exp?']== True, 'xp' ] = xp.loc[df['exp?'] == True, 'AskPrice']
xp = xp.loc[xp['exp?']== True]
xp= xp[['OptionKey', 'xp']]
xp

,OptionKey,xp
2019-11-15,TSLA2019-11-15P165.00,0.01
2019-11-15,TSLA2019-11-15P170.00,0.01
2019-11-15,TSLA2019-11-15P180.00,0.06
2019-11-15,TSLA2019-11-15P200.00,0.01
2019-11-15,TSLA2019-11-15P205.00,0.01
...,...,...
2019-12-13,TSLA2019-12-13P375.00,18.60
2019-12-13,TSLA2019-12-13P380.00,23.60
2019-12-13,TSLA2019-12-13P385.00,28.55
2019-12-13,TSLA2019-12-13P395.00,38.55


In [107]:
#mf = exp
#mf = mf[['OptionKey', 'xp']]
#mf = mf.dropna()
#mf
#msft_xp

,OptionKey,xp
3496,MSFT2019-06-28C100.00,34.90
3497,MSFT2019-06-28C105.00,29.25
3498,MSFT2019-06-28C110.00,24.25
3499,MSFT2019-06-28C114.00,20.95
3500,MSFT2019-06-28C117.00,17.05
...,...,...
74409,MSFT2019-06-14P134.00,1.58
74410,MSFT2019-06-14P135.00,2.59
74411,MSFT2019-06-14P136.00,3.60
74412,MSFT2019-06-14P137.00,4.70


In [16]:
df['xp']=df.OptionKey.map(xp.set_index('OptionKey')['xp'])
df

,OptionKey,ExpirationDate,AskPrice,BidPrice,PutCall,StrikePrice,UnderlyingPrice,DataDate,time delta,% out,...,LSTM OTM Cr Entry,RF OTM Cr Entry,Bid Price,delta,exp?,worth_it,Signal,execute,enter?,xp
2019-11-14,TSLA2019-11-15P135.00,2019-11-15,0.01,0.00,put,135.0,349.35,2019-11-14,1 days,-0.613568,...,0.0,0.0,0.0,1 days,False,False,-1,NaN,False,NaN
2019-11-13,TSLA2019-11-15P155.00,2019-11-15,0.05,0.00,put,155.0,346.11,2019-11-13,2 days,-0.552165,...,0.0,0.0,0.0,2 days,False,False,0,NaN,False,NaN
2019-11-13,TSLA2019-11-15P165.00,2019-11-15,0.01,0.00,put,165.0,346.11,2019-11-13,2 days,-0.523273,...,0.0,0.0,0.0,2 days,False,False,1,NaN,False,0.01
2019-11-15,TSLA2019-11-15P165.00,2019-11-15,0.01,0.00,put,165.0,352.17,2019-11-15,0 days,-0.531476,...,0.0,0.0,0.0,0 days,True,False,1,NaN,False,0.01
2019-11-15,TSLA2019-11-15P170.00,2019-11-15,0.01,0.00,put,170.0,352.17,2019-11-15,0 days,-0.517279,...,0.0,0.0,0.0,0 days,True,False,0,NaN,False,0.01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-11-19,TSLA2019-12-27P475.00,2019-12-27,118.35,114.00,put,475.0,359.52,2019-11-19,38 days,0.321206,...,0.0,11835.0,11400.0,38 days,False,False,1,NaN,False,NaN
2019-11-22,TSLA2019-12-27P475.00,2019-12-27,143.75,139.75,put,475.0,333.04,2019-11-22,35 days,0.426255,...,0.0,14375.0,13975.0,35 days,False,False,0,NaN,False,NaN
2019-11-25,TSLA2019-12-27P475.00,2019-12-27,140.50,136.35,put,475.0,336.34,2019-11-25,32 days,0.412261,...,0.0,0.0,13635.0,32 days,False,False,0,NaN,False,NaN
2019-12-18,TSLA2019-12-27P490.00,2019-12-27,97.60,96.30,put,490.0,393.15,2019-12-18,9 days,0.246344,...,9760.0,9760.0,9630.0,9 days,False,False,1,NaN,False,NaN


In [22]:
short_close = df.loc[(df['short_close']== True) & (df['Signal'] == 1)]
short_close['log'] = short_close['AskPrice'] - short_close['xp']
short_close_total = short_close['log'].sum()
print(short_close_total)

-55.53999999999999


<ipython-input-22-db0ecfdb9cad>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_close['log'] = short_close['AskPrice'] - short_close['xp']


In [23]:
short_medium = df.loc[(df['short_medium']== True) & (df['Signal'] == 1)]
short_medium['log'] = short_medium['AskPrice'] - short_medium['xp']
short_medium_total = short_medium['log'].sum()
print(short_medium_total)

3.41


<ipython-input-23-e8b5d44531af>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_medium['log'] = short_medium['AskPrice'] - short_medium['xp']


In [24]:
short_far = df.loc[(df['short_far']== True) & (df['Signal'] == 1)]
short_far['log'] = short_far['AskPrice'] - short_far['xp']
short_far_total = short_far['log'].sum()
print(short_far_total)

0.18000000000000002


<ipython-input-24-ac8845e429a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  short_far['log'] = short_far['AskPrice'] - short_far['xp']


In [27]:
long_close = df.loc[(df['long_close']== True) & (df['Signal'] == 1)]
long_close['log'] = long_close['AskPrice'] - long_close['xp']
long_close_total = long_close['log'].sum()
print(long_close_total)

-76.05999999999999


<ipython-input-27-75a269248bd1>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_close['log'] = long_close['AskPrice'] - long_close['xp']


In [29]:
long_far = df.loc[(df['long_far']== True) & (df['Signal'] == 1)]
long_far['log'] = long_far['AskPrice'] - long_far['xp']
long_far_total = long_far['log'].sum()
print(long_far_total)

0.5000000000000001


<ipython-input-29-f50bec28ebd4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  long_far['log'] = long_far['AskPrice'] - long_far['xp']


In [ ]:
long_medium = df.loc[(df['long_medium']== True) & (df['Signal'] == 1)]
long_medium['log'] = long_medium['AskPrice'] - long_medium['xp']
long_medium_total = long_medium['log'].sum()
print(long_medium_total)

In [66]:
short_close_negatives =(short_close.loc[(short_close.log < 0)])
short_close_negatives.log.mean()
# there are 14 times we were wrong

-6.436666666666667

In [70]:
short_close_positives =(short_close.loc[(short_close.log > 0)])
#short_close_positives.log.mean()
short_close_positives = short_close_positives.sort_values('log', ascending= False)
short_close_positives.head()

,OptionKey,ExpirationDate,AskPrice,BidPrice,PutCall,StrikePrice,UnderlyingPrice,DataDate,time delta,% out,LSTM Predicted Signal,RF Predicted Value,LSTM OTM Cr Entry,RF OTM Cr Entry,Bid Price,delta,exp?,worth_it,Signal,execute,enter?,xp,long_close,long_medium,long_far,short_close,short_medium,short_far,log
2019-12-09,TSLA2019-12-13P337.50,2019-12-13,4.05,3.90,put,337.5,339.53,2019-12-09,4 days,-0.005979,1,0,0.0,0.0,390.0,4 days,False,False,1,4.05,False,0.01,True,False,False,True,False,False,4.04
2019-12-03,TSLA2019-12-06P335.00,2019-12-06,3.75,3.55,put,335.0,336.20,2019-12-03,3 days,-0.003569,1,1,0.0,0.0,355.0,3 days,False,False,1,3.75,False,0.12,True,False,False,True,False,False,3.63
2019-12-06,TSLA2019-12-13P330.00,2019-12-13,3.05,2.91,put,330.0,335.89,2019-12-06,7 days,-0.017536,1,1,0.0,0.0,291.0,7 days,False,False,1,3.05,False,0.01,True,False,False,True,False,False,3.04
2019-12-09,TSLA2019-12-13P332.50,2019-12-13,2.30,2.22,put,332.5,339.53,2019-12-09,4 days,-0.020705,1,0,0.0,0.0,222.0,4 days,False,False,1,2.30,False,0.01,True,False,False,True,False,False,2.29
2019-12-10,TSLA2019-12-13P342.50,2019-12-13,1.75,1.69,put,342.5,348.84,2019-12-10,3 days,-0.018175,1,0,0.0,0.0,169.0,3 days,False,False,1,1.75,False,0.01,True,False,False,True,False,False,1.74


In [34]:
short_close.shape

(134, 29)

In [58]:
short_close = short_close.sort_values('log', ascending = True)
short_close = short_close.dropna()
short_close = short_close.dropindex()
short_close_max_premium = short_close['log'].tail()

In [60]:
short_close_max_loss = short_close['log'].head()
short_close_max_loss

2019-11-20   -10.75
2019-11-21   -10.37
2019-11-20    -9.15
2019-11-21    -8.31
2019-12-03    -0.02
Name: log, dtype: float64

In [57]:
print(f'the biggest loses were', short_close_max_loss)

the biggest loses were 2019-11-20   -10.75
2019-11-21   -10.37
2019-11-20    -9.15
2019-11-21    -8.31
2019-12-03    -0.02
Name: log, dtype: float64
